In [1]:
import cv2
import numpy as np

# === Load DNN Models ===
face_detector = cv2.FaceDetectorYN.create(
    "face_detection_yunet_2023mar.onnx", "", (320, 320),
    score_threshold=0.9, nms_threshold=0.3, top_k=5000
)
face_recognizer = cv2.FaceRecognizerSF.create("face_recognition_sface_2021dec.onnx", "")

# === Normalize Feature Embeddings ===
def normalize(vec):
    vec = np.squeeze(vec)  # (1,128) → (128,)
    norm = np.linalg.norm(vec)
    return vec / norm if norm > 0 else vec

# === Load Known Images ===
known_paths = [
    (r"C:\Users\veyadav\Downloads\dan.jpg", "dan"),
    (r"C:\Users\veyadav\Downloads\poorna.jpg", "poorna"),
    (r"C:\Users\veyadav\Downloads\mukesh.jpg", "mukesh")
]

known_features = []
known_labels = []

for img_path, label in known_paths:
    img = cv2.imread(img_path)
    if img is None:
        print(f"⚠️ Couldn't load: {img_path}")
        continue

    face_detector.setInputSize((img.shape[1], img.shape[0]))
    _, faces = face_detector.detect(img)

    if faces is not None and len(faces) > 0:
        aligned = face_recognizer.alignCrop(img, faces[0])
        aligned = cv2.resize(aligned, (112, 112))
        feature = face_recognizer.feature(aligned)
        known_features.append(normalize(feature))
        known_labels.append(label)
    else:
        print(f"⚠️ No face detected in: {img_path}")

if not known_features:
    raise ValueError("❌ No known faces loaded.")

# === Load Video ===
cap = cv2.VideoCapture(r'C:\Users\veyadav\Downloads\Drupal-Branching.mp4')
screen_width, screen_height = 1280, 720

cv2.namedWindow("DNN Face Recognition", cv2.WINDOW_NORMAL)
cv2.setWindowProperty("DNN Face Recognition", cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    face_detector.setInputSize((frame.shape[1], frame.shape[0]))
    _, faces = face_detector.detect(frame)

    if faces is not None:
        for i in range(faces.shape[0]):
            box = faces[i, :4].astype(int)
            aligned_face = face_recognizer.alignCrop(frame, faces[i])
            aligned_face = cv2.resize(aligned_face, (112, 112))
            test_feature = normalize(face_recognizer.feature(aligned_face))

            scores = [np.dot(test_feature, ref_feat) for ref_feat in known_features]
            best_index = int(np.argmax(scores))
            best_score = scores[best_index]
            best_label = known_labels[best_index]

            threshold = 0.36
            label = f"{best_label} ({best_score:.3f})" if best_score > threshold else "Not Found"

            if best_score > threshold:
                print(f"✅ Match: {best_label} @ {best_score:.3f} | Frame: {int(cap.get(cv2.CAP_PROP_POS_FRAMES))}")

            x, y, w, h = box
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, label, (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

    # === Resize frame to screen ===
    fh, fw = frame.shape[:2]
    scale = min(screen_width / fw, screen_height / fh)
    new_w, new_h = int(fw * scale), int(fh * scale)
    resized = cv2.resize(frame, (new_w, new_h))

    canvas = cv2.copyMakeBorder(
        resized,
        top=(screen_height - new_h) // 2,
        bottom=(screen_height - new_h + 1) // 2,
        left=(screen_width - new_w) // 2,
        right=(screen_width - new_w + 1) // 2,
        borderType=cv2.BORDER_CONSTANT,
        value=[0, 0, 0]
    )

    cv2.imshow("DNN Face Recognition", canvas)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

✅ Match: poorna @ 0.883 | Frame: 35
✅ Match: dan @ 0.433 | Frame: 35
✅ Match: mukesh @ 0.614 | Frame: 35
✅ Match: poorna @ 0.885 | Frame: 36
✅ Match: dan @ 0.429 | Frame: 36
✅ Match: mukesh @ 0.613 | Frame: 36
✅ Match: poorna @ 0.884 | Frame: 37
✅ Match: dan @ 0.429 | Frame: 37
✅ Match: mukesh @ 0.613 | Frame: 37
✅ Match: poorna @ 0.884 | Frame: 38
✅ Match: dan @ 0.429 | Frame: 38
✅ Match: mukesh @ 0.613 | Frame: 38
✅ Match: poorna @ 0.884 | Frame: 39
✅ Match: dan @ 0.429 | Frame: 39
✅ Match: mukesh @ 0.613 | Frame: 39
✅ Match: poorna @ 0.884 | Frame: 40
✅ Match: dan @ 0.429 | Frame: 40
✅ Match: mukesh @ 0.613 | Frame: 40
✅ Match: poorna @ 0.884 | Frame: 41
✅ Match: dan @ 0.429 | Frame: 41
✅ Match: mukesh @ 0.613 | Frame: 41
✅ Match: poorna @ 0.884 | Frame: 42
✅ Match: dan @ 0.429 | Frame: 42
✅ Match: mukesh @ 0.613 | Frame: 42
✅ Match: poorna @ 0.884 | Frame: 43
✅ Match: dan @ 0.429 | Frame: 43
✅ Match: mukesh @ 0.613 | Frame: 43
✅ Match: poorna @ 0.884 | Frame: 44
✅ Match: dan @ 0.42